# Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow_addons.metrics import F1Score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
import os

# Load Dataset

Set directories based on environment

In [ ]:
# >>>> google colab <<<<
# from google.colab import drive
# drive.mount('/content/drive')
# base_dir = '/content/drive/MyDrive/coin_classifier/'

# >>>> local <<<<
base_dir = '.'


dataset_dir = f'{base_dir}/ebay_images/'
model_dir = f'{base_dir}/saved_models/coin_classifier/'
tflite_dir = f'{base_dir}/saved_models/coin_classifier.tflite'

Create training and validation data generators

In [ ]:
batch_size = 64
img_dim = 224
img_size = (img_dim, img_dim)
img_shape = (img_dim, img_dim, 3)

train_datagen = ImageDataGenerator(
  height_shift_range=0.1,
  width_shift_range=0.1,
  rotation_range=45,
  brightness_range=(0.8, 1.2),
  rescale=1./255,
  validation_split=0.2,
)
test_datagen = ImageDataGenerator(
  rescale=1./255,
  validation_split=0.2,
)

train_generator = train_datagen.flow_from_directory(
  dataset_dir,
  target_size=img_size,
  batch_size=batch_size,
  subset='training',
  seed=1234,
)
test_generator = test_datagen.flow_from_directory(
  dataset_dir,
  target_size=img_size,
  batch_size=batch_size,
  subset='validation',
  shuffle=False,
  seed=1234,
)

labels = list(train_generator.class_indices.keys())
class_count = len(labels)

Display coin images

In [ ]:
fig = plt.figure(figsize=(16, 16))
X_batch, y_batch = train_generator.next()
for i in range(min(len(X_batch), 64)):
  fig.add_subplot(8, 8, i + 1)
  plt.imshow(X_batch[i])
  plt.axis('off')
  plt.title(labels[np.argmax(y_batch[i])][:14])

# Training

Create coin classifier model using ResNet50 architecture

In [ ]:
coin_classifier = ResNet50V2(classes=class_count, weights=None)
trainable_param_count = np.sum([np.prod(layer.shape) for layer in coin_classifier.trainable_weights])
nontrainable_param_count = np.sum([np.prod(layer.shape) for layer in coin_classifier.non_trainable_weights])
print('>>>> ResNet50 Model Summary <<<<')
print(f'Input shape: {coin_classifier.input.shape}')
print(f'Output shape: {coin_classifier.output.shape}')
print(f'Number of layers: {len(coin_classifier.layers)}')
print(f'Number of trainable parameters: {trainable_param_count}')
print(f'Number of non-trainable parameters: {nontrainable_param_count}')
print(f'Total number of parameters: {trainable_param_count+nontrainable_param_count}')

Train coin classifier

In [ ]:
coin_classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', F1Score(class_count)])
history = coin_classifier.fit(
  train_generator,
  callbacks=[
    EarlyStopping(monitor='val_accuracy', patience=20),
    ModelCheckpoint('checkpoint', monitor='val_accuracy', save_best_only=True),
  ],
  validation_data=test_generator,
  epochs=200,
)
coin_classifier = load_model('checkpoint')

Save training metrics

In [ ]:
if not os.path.isdir('metrics'):
  os.mkdir('metrics')

train_losses = history.history['loss']
train_accuracies = history.history['accuracy']
train_f1_scores = history.history['f1_score']

test_losses = history.history['val_loss']
test_accuracies = history.history['val_accuracy']
test_f1_scores = history.history['val_f1_score']

np.save('metrics/train_losses.npy', train_losses)
np.save('metrics/train_accuracies.npy', train_accuracies)
np.save('metrics/train_f1_scores.npy', train_f1_scores)

np.save('metrics/test_losses.npy', test_losses)
np.save('metrics/test_accuracies.npy', test_accuracies)
np.save('metrics/test_f1_scores.npy', test_f1_scores)

# Evaluation

Plot training and validation accuracies

In [ ]:
# plot losses
plt.plot(train_losses)
plt.plot(test_losses)
plt.title('Training and Testing Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'test'])
plt.show()

# plot accuracies
plt.plot(train_accuracies)
plt.plot(test_accuracies)
plt.title('Training and Testing Accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train', 'test'])
plt.show()

# plot f1 scores
plt.plot(train_f1_scores)
plt.plot(test_f1_scores)
plt.title('Training and Testing F1-Score')
plt.xlabel('epoch')
plt.ylabel('f1-score')
plt.legend(['train', 'test'])
plt.show()

Plot confusion matrix

In [ ]:
predictions = list(np.argmax(coin_classifier.predict(test_generator), axis=1))
true = list(test_generator.labels)

fig, ax = plt.subplots(figsize=(16, 16))
accuracy = round(accuracy_score(true, predictions) * 100, 2)
f1 = round(f1_score(true, predictions, average='macro'), 4)

conf_matrix = confusion_matrix(true, predictions)

ConfusionMatrixDisplay(conf_matrix, display_labels=labels).plot(
  xticks_rotation='vertical',
  cmap='plasma',
  ax=ax,
  colorbar=False,
)

plt.title(f'Test Set Confusion Matrix - Test Accuracy: {accuracy}% - Test F1: {f1}')
plt.show()

Print per-class metrics

In [ ]:
sort_by = 'recall'

precisions = precision_score(true, predictions, average=None)
recalls = recall_score(true, predictions, average=None)
f1_scores = f1_score(true, predictions, average=None)

if sort_by == 'precision':
  sorted_metric = sorted(enumerate(precisions), key=lambda x: x[1], reverse=True)
elif sort_by == 'recall':
  sorted_metric = sorted(enumerate(recalls), key=lambda x: x[1], reverse=True)
else:
   sorted_metric = sorted(enumerate(f1_scores), key=lambda x: x[1], reverse=True)

def get_mistakes(class_index):
  false_positives = sorted(enumerate(conf_matrix[:,class_index]), key=lambda x: x[1], reverse=True)
  false_negatives = sorted(enumerate(conf_matrix[class_index]), key=lambda x: x[1], reverse=True)
  false_positives = [(labels[i], count) for i, count in false_positives if count > 0 and i != class_index]
  false_negatives = [(labels[i], count) for i, count in false_negatives if count > 0 and i != class_index]
  return false_positives, false_negatives

for i, metric in sorted_metric:
  recall = metric if sort_by == 'recall' else recalls[i]
  precision = metric if sort_by == 'precision' else precisions[i]
  f1 = metric if sort_by == 'f1_score' else f1_scores[i]
  false_positives, false_negatives = get_mistakes(i)
  print(f'>>>> {labels[i]} <<<<')
  print(f'Precision: {round(precision * 100, 2)}%')
  print(f'Recall: {round(recall * 100, 2)}%')
  print(f'F1 Score: {round(f1, 4)}')
  print(f'False Positives: {false_positives}')
  print(f'False Negatives: {false_negatives}')
  print()

# Save Classifier

Save classifier normally

In [ ]:
coin_classifier.save(model_dir)

Save classifier using tflite format

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(model_dir)
tflite_model = converter.convert()
with open(tflite_dir, 'wb') as f:
  f.write(tflite_model)